In [ ]:
import figuregen
from figuregen import util
import pyexr
from pdf2image import convert_from_path
import IPython
from IPython.display import Image
import numpy as np
import cv2
import os

def loadpdf(pdfname):
    images = convert_from_path(pdfname, dpi=1000)
    return np.array(images[0])

def convert(pdfname):
    img = loadpdf(pdfname)
    cv2.imwrite(pdfname.replace('.pdf', '.png'), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def display(pdfname):
    img = convert(pdfname)
    IPython.display.display(Image(pdfname.replace('.pdf', '.png')))

In [ ]:
method_names = [ "Balance", "\\textbf{Ours}" ]

In [ ]:
def make_figure(scene_folder_vcm, scene_folder_bdpt, crop_vcm, crop_bdpt, filename, scene_name, exposure=0, show_method_names=True):
    vcm_images = [
        pyexr.read(os.path.join(scene_folder_vcm, folder, "render.exr"))
        for folder in [ "Vcm", "PdfRatioCombined" ]
    ]
    bdpt_images = [
        pyexr.read(os.path.join(scene_folder_bdpt, folder, "render.exr"))
        for folder in [ "BidirSplit", "PdfRatio" ]
    ]
    reference_image = pyexr.read(os.path.join(scene_folder_vcm, "reference.exr"))

    # Compute error values
    vcm_errors = [
        util.image.relative_mse(m, reference_image, 0.01)
        for m in vcm_images
    ]
    bdpt_errors = [
        util.image.relative_mse(m, reference_image, 0.01)
        for m in bdpt_images
    ]
    error_metric_name = "relMSE"

    def tonemap(img):
        return figuregen.JPEG(util.image.lin_to_srgb(util.image.exposure(img, exposure)), quality=80)

    # Reference image
    ref_grid = figuregen.Grid(1, 1)
    
    # Image and markers for the crops
    elem = ref_grid.get_element(0, 0)
    elem.set_image(tonemap(reference_image))
    elem.set_marker(crop_vcm.get_marker_pos(), crop_vcm.get_marker_size(), color=[255,255,255], linewidth_pt=1)
    elem.set_marker(crop_bdpt.get_marker_pos(), crop_bdpt.get_marker_size(), color=[255,255,255], linewidth_pt=1)

    # Set paddings and caption heights to align with the grid module
    layout = ref_grid.get_layout()
    layout.set_col_titles("top", fontsize=8, field_size_mm=2.8, offset_mm=0.25)
    layout.set_caption(height_mm=3, fontsize=8, offset_mm=0.25)
    layout.set_padding(right=1)

    def crop_error(img, crop):
        cropped_image = crop.crop(img)
        cropped_reference = crop.crop(reference_image)
        return util.image.relative_mse(cropped_image, cropped_reference, 0.01)

    crop_errors_vcm = [ crop_error(m, crop_vcm) for m in vcm_images ]
    crop_errors_bdpt = [ crop_error(m, crop_bdpt) for m in bdpt_images ]

    def error_string(index, array):
        value = f"${array[index]:.2f}$ "
        if index == 0:
            speedup = "$(1.00\\times)$"
        elif index == 1:
            speedup = "\\textbf{" + f"$(\\mathbf{{{array[index]/array[0]:.1f}}}\\times)$" + "}"
        else:
            speedup = f"$({array[index]/array[0]:.1f}\\times)$"
        return value + speedup

    # Comparison grid
    crop_grid = figuregen.Grid(num_cols=len(method_names)+1, num_rows=2)
    for col in range(0, len(method_names)):
        crop_grid.get_element(1, col).set_image(tonemap(crop_vcm.crop(vcm_images[col])))
        crop_grid.get_element(1, col).set_caption(error_string(col, vcm_errors))

        crop_grid.get_element(0, col).set_image(tonemap(crop_bdpt.crop(bdpt_images[col])))
        crop_grid.get_element(0, col).set_caption(error_string(col, bdpt_errors))

    crop_grid.get_element(1, 2).set_image(tonemap(crop_vcm.crop(reference_image)))
    crop_grid.get_element(1, 2).set_caption(f"{error_metric_name}")

    crop_grid.get_element(0, 2).set_image(tonemap(crop_bdpt.crop(reference_image)))
    crop_grid.get_element(0, 2).set_caption(f"{error_metric_name}")

    # Column titles
    crop_grid.set_col_titles("top", ["a) Balance heuristic", "b) \\textbf{Ours}", "c) Reference"])
    crop_grid.set_row_titles("left", ["BDPT w/ splitting", "VCM"])

    # Grid layout
    layout = crop_grid.get_layout()
    layout.set_padding(column=1, row=0.5, left=1)
    layout.set_col_titles("top", fontsize=8, field_size_mm=2.8, offset_mm=0.25)
    layout.set_row_titles("left", fontsize=8, field_size_mm=2.8, offset_mm=0.25)
    layout.set_caption(height_mm=3, fontsize=8, offset_mm=0.25)

    width_cm = 17.7
    figuregen.horizontal_figure([ref_grid, crop_grid], width_cm, filename, tex_packages=["{dfadobe}"])

In [ ]:
make_figure("./VcmExperiment/Results/IndirectRoom", "./BidirExperiment/Results-x10/IndirectRoom",
    util.image.Cropbox(top=700, left=1100, width=150, height=96, scale=10),
    util.image.Cropbox(top=300, left=10, width=150, height=96, scale=10),
    "Teaser.pdf", "\\textsc{Indirect Room}", exposure=0)
display("Teaser.pdf")